In [43]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
import math
from shapely.geometry import Point

import json
import math as m 

from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, Div, ColumnDataSource
from bokeh.palettes import brewer


from bokeh.plotting import figure, save
from bokeh.models.widgets import Panel, Tabs
from bokeh.io.doc import curdoc
from bokeh.layouts import widgetbox, row, column
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool, SingleIntervalTicker,\
    Slider, Button, Label, CategoricalColorMapper, Legend, Circle, CheckboxButtonGroup, Select, NumeralTickFormatter

import json
from bokeh.io import show, output_file
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox, gridplot
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.layouts import column, row
from matplotlib import cm
from bokeh.palettes import RdPu

# Bokeh Tab 1: Main Geoplot

## Filtering and merging data

In [2]:
def process_data():

        #__Importing geojson files
    # Importing the Neighbourhood boundaries data
    dr = os.getcwd()
    fdr = ''
    in_f = 'data/neighbourhoods.geojson'
    target = os.path.join(dr, in_f)
    df_neighbourhoods = gp.read_file(target)

    # Importing the Airbnb listings data with geopoints
    dr = os.getcwd()
    fdr = ''
    in_f = 'data/listings.geojson'
    target = os.path.join(dr, fdr, in_f)
    df_airbnb = gp.read_file(target)

    # Importing the Hotels data with geopoints
    dr = os.getcwd()
    fdr = ''
    in_f = 'data/sorted_hotels.geojson'
    target = os.path.join(dr, fdr, in_f)
    df_hotels = gp.read_file(target)
    
    # Importing the Hotels data with geopoints
    df_reviews = pd.read_csv("data/reviews.csv")


        # Encode the data so as they map onto the Web Mercator scale
    df_neighbourhoods = df_neighbourhoods.to_crs(epsg=3857)
    df_airbnb = df_airbnb.to_crs(epsg=3857)
    df_hotels = df_hotels.to_crs(epsg=3857)

        #remove neighbourhoods outside manhattan
    df_neighbourhoods = df_neighbourhoods[df_neighbourhoods.neighbourhood_group == 'Manhattan']
    df_airbnb = df_airbnb[df_airbnb.neighbourhood_group == 'Manhattan']

        #____Number of  Airbnbs per Neighbourhood

    # Merging reviews with neighbourhoods on Airbnb ID
    df_man = df_airbnb.filter(['neighbourhood','id'], axis=1)
    df_reviews_count = df_reviews.rename(columns={'listing_id': 'id'})
    df_id = pd.merge(df_reviews_count, df_man, on='id')

    # Extracting year from data
    df_id['date'] = pd.to_datetime(df_id['date'])
    df_id['year'] = df_id['date'].dt.year
    df_id = df_id.drop('date', axis=1)

    # Counting the number (amount) of airbnbs in each neighbourhood
    df_id.loc[:, 'counter'] = 1
    df_id = df_id.drop_duplicates(subset=None, keep='first', inplace=False)
    df_id = df_id.drop('id', axis=1)
    df_id = df_id.drop('counter', axis=1)

    #groupby
    df_id_1 = df_id.groupby(df_id.columns.tolist(),as_index=False).size().reset_index(name="amount")


    # merging yearly airbnbs with neighbourhoods
    dtf = pd.merge(df_neighbourhoods, df_id_1, on='neighbourhood')
    dtf = dtf.drop('neighbourhood_group', axis=1)

    # setting line around patches of neightbourhoods
    dtf.loc[:, 'line'] = 1

        #____Finding First Review per Airbnb

    # Creating new dataframe with first review of each airbnb
    df_reviews = df_reviews.rename(columns={'listing_id': 'id'})
    df_reviews['date'] = pd.to_datetime(df_reviews['date'])
    df_reviews = pd.DataFrame(df_reviews.groupby('id')['date'].min()) #first review of each listing

    # Creating new dataframe with first review data and location
    airbnb = pd.merge(df_airbnb, df_reviews, on = 'id')
    airbnb['year'] = airbnb['date'].dt.year
    
    return airbnb, df_hotels, dtf

## Set up data for plotting

In [45]:
airbnb, df_hotels, dtf = process_data()


#_________________________Get X and Y coordinates of Airbnbs and Hotels______________________________

#create function to get x and y coordinates
def getPointCoords(row, geom, coord_type):

    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

airbnb['x'] = airbnb.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
airbnb['y'] = airbnb.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)
    
df_hotels['x'] = df_hotels.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
df_hotels['y'] = df_hotels.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)


#Create copied dataframe for airbnbs and drop geometry column to plot as points
airbnb_df = airbnb.drop('geometry', axis = 1).copy()
hotels_df = df_hotels.drop('geometry', axis = 1).copy()

# Call column data source for hotels, airbnbs and the shape of manhattan
neighbourhood_source = GeoJSONDataSource(geojson = dtf.to_json())
airbnb_source = ColumnDataSource(airbnb_df)
hotels_source = ColumnDataSource(hotels_df)

# Bokeh Tab 2: Density of Airbnbs around Hotels in Most Affected Neighbourhoods

In [6]:
#import data
airbnb_500 = airbnb_df
airbnb_hotels = pd.read_csv('data/hotels_airbnb.csv')
hotels = pd.read_csv("data/Hotel Addressess.csv")
affected_hotels = pd.read_csv("data/affected_hotels.csv")

## Affected Hotels

In [7]:
#top rated hotel per neighbourhood
affected_cords = hotels[(hotels.doc_id == 'the plaza') | (hotels.doc_id == 'millenium hilton') 
                        | (hotels.doc_id == '414 hotel') | (hotels.doc_id == 'the standard new york')
                        | (hotels.doc_id == 'the carlyle a rosewood hotel')
                        | (hotels.doc_id == 'duane street hotel') | (hotels.doc_id == 'hotel beacon')
                        | (hotels.doc_id == 'the sylvan guest house') 
                        | (hotels.doc_id == 'the bowery hotel') | (hotels.doc_id == 'astor on the park')
                        | (hotels.doc_id == 'hotel deauville')
                       ]

#merge to get long, lat, xcoords and ycoords in same dataframe
affected = pd.merge(affected_cords, affected_hotels, on = 'doc_id')

#assigning each hotel to its respective row
plaza_hotel = affected[affected.doc_id == 'the plaza']
mill_hilton = affected[(affected.doc_id == 'millenium hilton')]
hotel_414 = affected[(affected.doc_id == '414 hotel')]
the_standard = affected[(affected.doc_id == 'the standard new york')]
the_carlyle = affected[(affected.doc_id == 'the carlyle a rosewood hotel')]
duane_street = affected[(affected.doc_id == 'duane street hotel')]
hotel_beacon = affected[(affected.doc_id == 'hotel beacon')]
the_sylvan = affected[(affected.doc_id == 'the sylvan guest house')]
the_bowery = affected[(affected.doc_id == 'the bowery hotel')]
astor_park = affected[(affected.doc_id == 'astor on the park')]
hotel_deauville = affected[(affected.doc_id ==  'hotel deauville')]

## Plaza Hotel

In [8]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(plaza_hotel['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(plaza_hotel['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [9]:
plaza = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
plaza = pd.merge(plaza, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## Mill Hilton

In [10]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(mill_hilton['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(mill_hilton['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [12]:
hilton = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
hilton = pd.merge(hilton, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## For 414 Hotel 

In [13]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(hotel_414['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(hotel_414['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [14]:
hot414 = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
hot414 = pd.merge(hot414, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## The Standard

In [15]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(the_standard['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(the_standard['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [16]:
standard = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
standard = pd.merge(standard, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## The Carlyle

In [17]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(the_carlyle['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(the_carlyle['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [18]:
carlyle = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
carlyle = pd.merge(carlyle, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## Duane Street

In [19]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(duane_street['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(duane_street['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [21]:
duane = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
duane = pd.merge(duane, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## Hotel Beacon

In [22]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(hotel_beacon['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(hotel_beacon['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [23]:
beacon = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
beacon = pd.merge(beacon, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## The Sylvian Guest House

In [24]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(the_sylvan['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(the_sylvan['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [25]:
sylvan = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
sylvan = pd.merge(sylvan, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## The Browery Hotel

In [26]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(the_bowery['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(the_bowery['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [27]:
bowery = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
bowery = pd.merge(bowery, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## The Astor on the Park

In [28]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(astor_park['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(astor_park['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [29]:
astor = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
astor = pd.merge(astor, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## Hotel Deauville

In [30]:
# transform list into data frame
def distance():
    data = pd.read_csv("data/hotels_airbnb.csv", usecols = ['host_id', 'latitude', 'longitude'])
    lat = data['latitude']
    lon = data['longitude']
    dist = []
    radius = 6371
    
    for i in range(len(lat)):
        phi1 = m.radians(hotel_deauville['lat'])
        phi2 = m.radians(lat[i])
        lambda1 = m.radians(hotel_deauville['long'])
        lambda2 = m.radians(lon[i])
        dlat = phi2 - phi1
        dlon = lambda2 -lambda1
        a = m.sqrt(m.sin(dlat/2)**2+m.cos(phi1)*m.cos(phi2)*m.sin(dlon/2)**2) #use of Haversine formula
        d = 2*radius*np.arcsin(a)
        dist.append(d)
    return dist

distance()
distance = distance()
airbnb_hotels['distance'] = distance

In [31]:
deauville = airbnb_hotels[airbnb_hotels.distance < 0.5 ].copy()
deauville = pd.merge(deauville, airbnb_500, how = 'left', left_on = 'id', right_on = 'id')

## Import Additional Libraries

In [32]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
from bokeh.models import HoverTool, Legend, ColumnDataSource

In [46]:
affected_cdf = ColumnDataSource(affected)
#legend = ("The Plaza", hotels_plot)

# Format the tooltip
tooltips = [('Hotel','@doc_id'),
            ('Neighbourhood', '@neighbourhood'),
            ('Address', '@hotel_url_x'),
            ('Airbnbs under 500m', '@no_airbnbs')   
           ]

# Bokeh Tab 3: Graphical Analysis of Most Affected Neighbourhoods

## Reimport original data

In [34]:
# Importing the neighbourhoods with geopoints
dr = os.getcwd()
fdr = ''
in_f = 'data/neighbourhoods.geojson'
target = os.path.join(dr, in_f)
df_neighbourhoods1 = gp.read_file(target)

# Importing the Airbnb listings data with geopoints
dr = os.getcwd()
fdr = ''
in_f = 'data/listings.geojson'
target = os.path.join(dr, fdr, in_f)
df_airbnb1 = gp.read_file(target)

# Importing the Hotels data with geopoints
dr = os.getcwd()
fdr = ''
in_f = 'data/sorted_hotels.geojson'
target = os.path.join(dr, fdr, in_f)
df_hotels1 = gp.read_file(target)

# Importing the Hotels data with geopoints
df_reviews1 = pd.read_csv("data/reviews.csv")

## Filter dataframes and merge

In [35]:
#remove neighbourhoods outside manhattan
df_neighbourhoods1 = df_neighbourhoods1[df_neighbourhoods1.neighbourhood_group == 'Manhattan']

# remove airbnbs outside manhattan
df_airbnb1 = df_airbnb1[df_airbnb1.neighbourhood_group == 'Manhattan'] 

In [36]:
# Import reviews and rename listing_id to match with airbnb dataframe
df_reviews1 = df_reviews1.rename(columns= {"listing_id":"id"})

# Filter reviews to only display the first review date for each airbnb
df_reviews1 = pd.DataFrame(df_reviews1.groupby('id')['date'].min())

# Merge airbnb with reviews. This does remove airbnbs that have never had any reviews, however. 
df_airbnb1 = pd.merge(df_airbnb1, df_reviews1, on='id')

# Extract year and month
df_airbnb1['date'] = pd.to_datetime(df_airbnb1['date'])

# Drop geometry
df_airbnb1 = df_airbnb1.drop('geometry', axis=1)

# Add counter column for counting the different filters later
df_airbnb1.loc[:, 'counter'] = 1 

# Sort datetime values from low to high
df_airbnb1 = df_airbnb1.sort_values('date')

## Create Initial Filter Objects for Airbnb Dataframe

In [37]:
# Listing Type
entire = df_airbnb1[df_airbnb1.room_type == 'Entire home/apt']
room = df_airbnb1[df_airbnb1.room_type == 'Private room'] 

# Availability less or more than 100 days
a_less = df_airbnb1[df_airbnb1.availability_365 < 100]
a_more = df_airbnb1[df_airbnb1.availability_365 >= 100]

# Minimum stay less or more than 30 days
stay_less = df_airbnb1[df_airbnb1.minimum_nights < 30]

# Most competitive airbnbs
most_comp = df_airbnb1[(df_airbnb1.room_type == 'Entire home/apt') & (df_airbnb1.availability_365 >= 100)
                     & (df_airbnb1.minimum_nights < 30)]

## Specify Most Affected Neighbourhoods

In [38]:
#Entire

entire_midtown = entire[entire.neighbourhood == 'Midtown']
entire_financial = entire[entire.neighbourhood == 'Financial District']
entire_hells = entire[entire.neighbourhood == 'Hell\'s'' Kitchen']
entire_east = entire[entire.neighbourhood == 'East Village']
entire_upper = entire[entire.neighbourhood == 'Upper East Side']

#Private rooms

private_midtown = room[room.neighbourhood == 'Midtown']
private_financial = room[room.neighbourhood == 'Financial District']
private_hells = room[room.neighbourhood == 'Hell\'s'' Kitchen']
private_east = room[room.neighbourhood == 'East Village']
private_upper = room[room.neighbourhood == 'Upper East Side']

#Availability less than 30 days

aless_midtown = a_less[a_less.neighbourhood == 'Midtown']
aless_financial = a_less[a_less.neighbourhood == 'Financial District']
aless_hells = a_less[a_less.neighbourhood == 'Hell\'s'' Kitchen']
aless_east = a_less[a_less.neighbourhood == 'East Village']
aless_upper = a_less[a_less.neighbourhood == 'Upper East Side']

#Availability more than 30 days

amore_midtown = a_more[a_more.neighbourhood == 'Midtown']
amore_financial = a_more[a_more.neighbourhood == 'Financial District']
amore_hells = a_more[a_more.neighbourhood == 'Hell\'s'' Kitchen']
amore_east = a_more[a_more.neighbourhood == 'East Village']
amore_upper = a_more[a_more.neighbourhood == 'Upper East Side']

#Minimum stay less than 30 days

stay_midtown = stay_less[stay_less.neighbourhood == 'Midtown']
stay_financial = stay_less[stay_less.neighbourhood == 'Financial District']
stay_hells = stay_less[stay_less.neighbourhood == 'Hell\'s'' Kitchen']
stay_east = stay_less[stay_less.neighbourhood == 'East Village']
stay_upper = stay_less[stay_less.neighbourhood == 'Upper East Side']

# Most competitive airbnbs

comp_midtown = most_comp[most_comp.neighbourhood == 'Midtown']
comp_financial = most_comp[most_comp.neighbourhood == 'Financial District']
comp_hells = most_comp[most_comp.neighbourhood == 'Hell\'s'' Kitchen']
comp_east = most_comp[most_comp.neighbourhood == 'East Village']
comp_upper = most_comp[most_comp.neighbourhood == 'Upper East Side']

## Add cumulative sum column by each filter

In [39]:
# Add cumulative sum of airbnbs by each 
entire_midtown.loc[:,'cumsum'] = entire_midtown.groupby('neighbourhood')['counter'].cumsum()
entire_financial.loc[:,'cumsum'] = entire_financial.groupby('neighbourhood')['counter'].cumsum()
entire_hells.loc[:,'cumsum'] = entire_hells.groupby('neighbourhood')['counter'].cumsum()
entire_east.loc[:,'cumsum'] = entire_east.groupby('neighbourhood')['counter'].cumsum()
entire_upper.loc[:,'cumsum'] = entire_upper.groupby('neighbourhood')['counter'].cumsum()

private_midtown.loc[:,'cumsum'] = private_midtown.groupby('neighbourhood')['counter'].cumsum()
private_financial.loc[:,'cumsum'] = private_financial.groupby('neighbourhood')['counter'].cumsum()
private_hells.loc[:,'cumsum'] = private_hells.groupby('neighbourhood')['counter'].cumsum()
private_east.loc[:,'cumsum'] = private_east.groupby('neighbourhood')['counter'].cumsum()
private_upper.loc[:,'cumsum'] = private_upper.groupby('neighbourhood')['counter'].cumsum()

# Add cumulative sum of airbnbs by each 
aless_midtown.loc[:,'cumsum'] = aless_midtown.groupby('neighbourhood')['counter'].cumsum()
aless_financial.loc[:,'cumsum'] = aless_financial.groupby('neighbourhood')['counter'].cumsum()
aless_hells.loc[:,'cumsum'] = aless_hells.groupby('neighbourhood')['counter'].cumsum()
aless_east.loc[:,'cumsum'] = aless_east.groupby('neighbourhood')['counter'].cumsum()
aless_upper.loc[:,'cumsum'] = aless_upper.groupby('neighbourhood')['counter'].cumsum()

# Add cumulative sum of airbnbs by each 
amore_midtown.loc[:,'cumsum'] = amore_midtown.groupby('neighbourhood')['counter'].cumsum()
amore_financial.loc[:,'cumsum'] = amore_financial.groupby('neighbourhood')['counter'].cumsum()
amore_hells.loc[:,'cumsum'] = amore_hells.groupby('neighbourhood')['counter'].cumsum()
amore_east.loc[:,'cumsum'] = amore_east.groupby('neighbourhood')['counter'].cumsum()
amore_upper.loc[:,'cumsum'] = amore_upper.groupby('neighbourhood')['counter'].cumsum()

# Add cumulative sum of airbnbs by each 
stay_midtown.loc[:,'cumsum'] = stay_midtown.groupby('neighbourhood')['counter'].cumsum()
stay_financial.loc[:,'cumsum'] = stay_financial.groupby('neighbourhood')['counter'].cumsum()
stay_hells.loc[:,'cumsum'] = stay_hells.groupby('neighbourhood')['counter'].cumsum()
stay_east.loc[:,'cumsum'] = stay_east.groupby('neighbourhood')['counter'].cumsum()
stay_upper.loc[:,'cumsum'] = stay_upper.groupby('neighbourhood')['counter'].cumsum()

# Add cumulative sum of airbnbs by each 
comp_midtown.loc[:,'cumsum'] = comp_midtown.groupby('neighbourhood')['counter'].cumsum()
comp_financial.loc[:,'cumsum'] = comp_financial.groupby('neighbourhood')['counter'].cumsum()
comp_hells.loc[:,'cumsum'] = comp_hells.groupby('neighbourhood')['counter'].cumsum()
comp_east.loc[:,'cumsum'] = comp_east.groupby('neighbourhood')['counter'].cumsum()
comp_upper.loc[:,'cumsum'] = comp_upper.groupby('neighbourhood')['counter'].cumsum()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Change Dataframes to dictionaries for plotting in bokeh

In [41]:
entire_midtown = entire_midtown.to_dict('list')
entire_financial = entire_financial.to_dict('list')
entire_hells = entire_hells.to_dict('list')
entire_east = entire_east.to_dict('list')
entire_upper = entire_upper.to_dict('list')

private_midtown = private_midtown.to_dict('list')
private_financial = private_financial.to_dict('list')
private_hells = private_hells.to_dict('list')
private_east = private_east.to_dict('list')
private_upper = private_upper.to_dict('list')

aless_midtown = aless_midtown.to_dict('list')
aless_financial = aless_financial.to_dict('list')
aless_hells = aless_hells.to_dict('list')
aless_east = aless_east.to_dict('list')
aless_upper = aless_upper.to_dict('list')

amore_midtown = amore_midtown.to_dict('list')
amore_financial = amore_financial.to_dict('list')
amore_hells = amore_hells.to_dict('list')
amore_east = amore_east.to_dict('list')
amore_upper = amore_upper.to_dict('list')

stay_midtown = stay_midtown.to_dict('list')
stay_financial = stay_financial.to_dict('list')
stay_hells = stay_hells.to_dict('list')
stay_east = stay_east.to_dict('list')
stay_upper = stay_upper.to_dict('list')

comp_midtown = comp_midtown.to_dict('list')
comp_financial = comp_financial.to_dict('list')
comp_hells = comp_hells.to_dict('list')
comp_east = comp_east.to_dict('list')
comp_upper = comp_upper.to_dict('list')

# Create Final Plot with Tabs

In [47]:
output_file('Competitive Market Analysis for Manhattan Hotel Association.html')

#----------------------------------COLOR MAP-----------------------------


# Define color palettes
palette = brewer['PuBuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1600)

# # Define custom tick labels for color bar.
# tick_labels = {0:'0', 250:'250',500:'500',750:'750', 1000:'1000', 1250:'1250',1500:'1500'}


# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper,
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal')


#______________________________________Tab 1___________________________________

# Set up figure
p1 = figure(title = 'Number of Airbnbs by Neighbourhoods in Manhattan - 2009 to 2019', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'above',
           tools = 'pan, wheel_zoom, reset')
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None


# Set the label on the top left corner to indicate the current year the data is presenting
label = Label(text = 'slider.value', text_font_size='60pt', text_color='#8a8a8a')
p1.add_layout(label)


#---------------------------Create Slider---------------------------

# Make a slider object to toggle the year shown
slider = Slider(start = 2009, end = 2019, 
                value = 2009, step = 1, title = 'year')

# This callback triggers the filter when the slider changes
callback1 = CustomJS(args = dict(source=neighbourhood_source), 
                    code = """source.change.emit();""")

slider.js_on_change('value', callback1)

# This callback triggers the filter when the slider changes
callback2 = CustomJS(args = dict(source=airbnb_source), 
                    code = """source.change.emit();""")

slider.js_on_change('value', callback2)


# Creates custom filter that selects the rows of the year based on the value in the slider
custom_filter_1 = CustomJSFilter(args = dict(slider = slider, 
                                           source = neighbourhood_source), 
                               code = '''
var indices = [];
for (var i = 0; i < source.get_length(); i++){
 if (source.data['year'][i] == slider.value){
 indices.push(true);
 } else {
 indices.push(false);
 }
}
return indices;
''')


# Creates custom filter that selects the rows of the year based on the value in the slider
custom_filter_2 = CustomJSFilter(args = dict(slider = slider, 
                                           source = airbnb_source), 
                               code = '''
var indices = [];
for (var i = 0; i < source.get_length(); i++){
 if (source.data['year'][i] == slider.value){
 indices.push(true);
 } else {
 indices.push(false);
 }
}
return indices;
''')

# Uses custom_filter to determine which set of airbnbs are visible
view1 = CDSView(source = neighbourhood_source, filters = [custom_filter_1])
view2 = CDSView(source = airbnb_source, filters = [custom_filter_2])


# Add patch renderer to figure.
man = p1.patches('xs','ys', source = neighbourhood_source, line_color = {'field' :'line'}, 
                line_alpha = 1,
                fill_color = {'field' :'amount', 'transform' : color_mapper},
                fill_alpha = 0.5, view = view1)

man1 = p1.patches('xs','ys', source = neighbourhood_source, line_color = 'black', 
                line_alpha = 1, fill_color=None)

# Plot airbnbs based on x and y coordinates
p1.circle('x', 'y', source = airbnb_source, color = 'orange', size=2, 
         alpha = 1, view = view2)

# Create object to plot hotels based on x and y coordinates, call in hovertool
p1.square('x', 'y', source = hotels_source, color = 'royalblue', 
                  size = 5, alpha = 0.5)

# Create hover tool for neighbourhood

p1.add_tools(HoverTool(renderers = [man],
                      tooltips = [('neighbourhood','@neighbourhood'),
                               ('Number of Airbnbs','@amount')]))

# Specify layout
p1.add_layout(color_bar, 'below')
p1.axis.visible = False


# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p1, widgetbox(slider))

#_______________________________________Tab 2_____________________________________


#set up figure
p2 = figure(title = 'Airbnbs within 500m of most popular hotels 2019',
            plot_height = 700, plot_width = 800,
            x_range=(-8250000,-8220000), y_range=(4975000,4982000),
            x_axis_type="mercator", y_axis_type="mercator")
p2.add_tile(CARTODBPOSITRON_RETINA)



# Add square representing each player
p2.square(x='x',y='y', source=affected_cdf,
         color='royalblue', selection_color='deepskyblue',
         nonselection_color='lightgray', nonselection_alpha=0.3)

# Configure a renderer to be used upon hover
hover_glyph = p2.circle(x='x', y='y', source=affected_cdf,
                         size=50, alpha=0,
                         hover_fill_color='black', hover_alpha=0.5)


#plot airbnbs around each hotel
plaza_plot = p2.circle(x = plaza['x'], y = plaza['y'], color = 'orange', alpha = 0.3)
hilton_plot = p2.circle(x = hilton['x'], y = hilton['y'], color = 'orange', alpha = 0.3)
hotel414_plot = p2.circle(x = hot414['x'], y = hot414['y'], color = 'orange', alpha = 0.3)
standard_plot = p2.circle(x = standard['x'], y = standard['y'], color = 'orange', alpha = 0.3)
carlyle_plot = p2.circle(x = carlyle['x'], y = carlyle['y'], color = 'orange', alpha = 0.3)
duane_plot = p2.circle(x = duane['x'], y = duane['y'], color = 'orange', alpha = 0.3)
beacon_plot = p2.circle(x = beacon['x'], y = beacon['y'], color = 'orange', alpha = 0.3)
sylvan_plot = p2.circle(x = sylvan['x'], y = sylvan['y'], color = 'orange', alpha = 0.3)
bowery_plot = p2.circle(x = bowery['x'], y = bowery['y'], color = 'orange', alpha = 0.3)
astor_plot = p2.circle(x = astor['x'], y = astor['y'], color = 'orange', alpha = 0.3)
deauville_plot = p2.circle(x = deauville['x'], y = deauville['y'], color = 'orange', alpha = 0.3)


p2.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
p2.axis.visible = False


#_______________________________________Tab 3_________________________________________

# --------------------------------Entire Homes-----------------------------

color = brewer['Set2'][5]

p3 = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p3.title.text = 'No. of entire Airbnb homes/apts by year (click on legend entries to hide)'

for data, name, color in zip([entire_midtown, entire_financial, entire_hells, entire_east, entire_upper],
                             ["Midtown", "Financial District", "Hells Kitchen", "East Village", "Upper East Side"],
                             color):
    df = pd.DataFrame(data)
    #df['date'] = pd.to_datetime(df['date'])
    p3.line(df['date'], df['cumsum'], line_width=2, color=color, alpha=0.8, legend=name)

p3.legend.location = "top_left"
p3.legend.click_policy="hide"
p3.legend.label_text_font_size = '6pt'

# --------------------------------Private Rooms-----------------------------

color = brewer['Set2'][5]

p4 = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p4.title.text = 'No. of private Airbnb rooms by year (click on legend entries to hide)'

for data, name, color in zip([private_midtown, private_financial, private_hells, private_east, private_upper],
                             ["Midtown", "Financial District", "Hells Kitchen", "East Village", "Upper East Side"],
                             color):
    df = pd.DataFrame(data)
    p4.line(df['date'], df['cumsum'], line_width=2, color=color, alpha=0.8, legend=name)

p4.legend.location = "top_left"
p4.legend.click_policy="hide"
p4.legend.label_text_font_size = '6pt'

# -----------------------Less than 100 days availability--------------------

color = brewer['Set2'][5]

p5 = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p5.title.text = 'No. of Airbnbs with <100 days availability by year (click on legend entries to hide)'

for data, name, color in zip([aless_midtown, aless_financial, aless_hells, aless_east, aless_upper],
                             ["Midtown", "Financial District", "Hells Kitchen", "East Village", "Upper East Side"],
                             color):
    df = pd.DataFrame(data)
    p5.line(df['date'], df['cumsum'], line_width=2, color=color, alpha=0.8, legend=name)

p5.legend.location = "top_left"
p5.legend.click_policy="hide"
p5.legend.label_text_font_size = '6pt'

# -----------------------More than 100 days availability--------------------

color = brewer['Set2'][5]

p6 = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p6.title.text = 'No. of Airbnbs with >100 days availability by year (click on legend entries to hide)'

for data, name, color in zip([amore_midtown, amore_financial, amore_hells, amore_east, amore_upper],
                             ["Midtown", "Financial District", "Hells Kitchen", "East Village", "Upper East Side"],
                             color):
    df = pd.DataFrame(data)
    p6.line(df['date'], df['cumsum'], line_width=2, color=color, alpha=0.8, legend=name)

p6.legend.location = "top_left"
p6.legend.click_policy="hide"
p6.legend.label_text_font_size = '6pt'

# -----------------------Minimum stay less than 30 days--------------------

color = brewer['Set2'][5]

p7 = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p7.title.text = 'No. of Airbnbs with <30 days min stay by year (click on legend entries to hide)'

for data, name, color in zip([stay_midtown, stay_financial, stay_hells, stay_east, stay_upper],
                             ["Midtown", "Financial District", "Hells Kitchen", "East Village", "Upper East Side"],
                             color):
    df = pd.DataFrame(data)
    p7.line(df['date'], df['cumsum'], line_width=2, color=color, alpha=0.8, legend=name)

p7.legend.location = "top_left"
p7.legend.click_policy="hide"
p7.legend.label_text_font_size = '6pt'


# -----------------------Most competitive Airbnbs--------------------

color = brewer['Set2'][5]

p8 = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p8.title.text = 'No. of most competitive Airbnbs by year (click on legend entries to hide)'

for data, name, color in zip([comp_midtown, comp_financial, comp_hells, comp_east, comp_upper],
                             ["Midtown", "Financial District", "Hells Kitchen", "East Village", "Upper East Side"],
                             color):
    df = pd.DataFrame(data)
    p8.line(df['date'], df['cumsum'], line_width=2, color=color, alpha=0.8, legend=name)

p8.legend.location = "top_left"
p8.legend.click_policy="hide"
p8.legend.label_text_font_size = '6pt'

# make a grid

grid = gridplot([p3, p4, p5, p6, p7, p8], ncols=2, plot_width=600, plot_height=200)


# -----------------------Create tabs and save output file--------------------


tab1 = Panel(child=layout,title="Number of Airbnbs by neighbourhood")
tab2 = Panel(child=p2,title="Density of Airbnbs around most popular hotels")
tab3 = Panel(child=grid,title="Filtered growth of Airbnbs in Manhattans' most affected neighbourhoods")


tabs = Tabs(tabs=[ tab1, tab2, tab3 ])

curdoc().add_root(tabs)

show(tabs)

save(tabs)

'/home/ubuntu/Competitive Market Analysis for Manhattan Hotel Association.html'